# ADK Application Testing

This notebook demonstrates how to test an ADK (Agent Development Kit) application.
It covers both local and remote testing, both with Agent Engine and Cloud Run.

> **Note**: This notebook assumes that the agent files are stored in the `app` folder. If your agent files are located in a different directory, please update all relevant file paths and references accordingly.

<img src="https://github.com/GoogleCloudPlatform/agent-starter-pack/blob/main/docs/images/adk_logo.png?raw=true" width="400">



## Install dependencies

In [ ]:
!pip install google-cloud-aiplatform --upgrade

### Import libraries

In [26]:
import json

import requests
import vertexai

### Initialize Vertex AI Client

In [27]:
# Initialize the Vertex AI client
LOCATION = "europe-west4"

client = vertexai.Client(
    location=LOCATION,
)

## If you are using Agent Engine
See more documentation at [Agent Engine Overview](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview)

### Remote Testing

In [ ]:
# Replace PROJECT_ID and ENGINE_ID with your values
ENGINE_ID = "8096944364430819328"
AGENT_ENGINE_ID = f"projects/rocketech-de-pgcp-sandbox/locations/{LOCATION}/reasoningEngines/{ENGINE_ID}"

# Get the existing agent engine
remote_agent_engine = client.agent_engines.get(name=AGENT_ENGINE_ID)
print(f"Retrieved agent engine: {remote_agent_engine.api_resource.name}")

In [ ]:
remote_session = await remote_agent_engine.async_create_session(user_id="shenghua.he@rocketech.co.uk")
print(remote_session)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

async for event in remote_agent_engine.async_stream_query(
    user_id="shenghua.he@rocketech.co.uk",
    session_id=remote_session["id"],
    message="Top 5 Most Sold Products including product category from the past 12 weeks",
):


    # Your existing code
    if 'function_call' in event['content']['parts'][0]:
        part = event['content']['parts'][0]
        print('function call name:', part['function_call']['name'])
        print('function call question:', part['function_call']['args']['question'])
        print('\n----------')
    elif 'function_response' in event['content']['parts'][0]:
        part = event['content']['parts'][0]
        print('function response status:', part['function_response']['response']['status'])
        print('function response results query:', part['function_response']['response']['results']['query'])
        print('function response results sql:', part['function_response']['response']['results']['generated_sql'])
        print('function response results data:', part['function_response']['response']['results']['data'])

        # New code to add visualization
        data = part['function_response']['response']['results']['data']

        df = pd.DataFrame(json.loads(data))

        print('\n----------')
    elif 'text' in event['content']['parts'][0]:
        part = event['content']['parts'][0]
        print('Final text:', part['text'])
        print('\n----------')


In [ ]:
df

### Local Testing

You can import directly the AgentEngineApp class within your environment.
To run the agent locally, follow these steps:
1. Make sure all required packages are installed in your environment
2. The recommended approach is to use the same virtual environment created by the 'uv' tool
3. You can set up this environment by running 'make install' from your agent's root directory
4. Then select this kernel (.venv folder in your project) in your Jupyter notebook to ensure all dependencies are available

In [ ]:
# Uncomment the following lines if you're not using the virtual environment created by uv
# import sys
# sys.path.append("../")

In [ ]:
from app.agent import root_agent
from app.agent_engine_app import AgentEngineApp

agent_engine = AgentEngineApp(agent=root_agent)

In [ ]:
async for event in agent_engine.async_stream_query(message="hi!", user_id="test"):
    print(event)

## If you are using Cloud Run

#### Remote Testing

For more information about authenticating HTTPS requests to Cloud Run services, see:
[Cloud Run Authentication Documentation](https://cloud.google.com/run/docs/triggering/https-request)

Remote testing involves using a deployed service URL instead of localhost.

Authentication is handled using GCP identity tokens instead of local credentials.

In [ ]:
ID_TOKEN = get_ipython().getoutput("gcloud auth print-identity-token -q")[0]

In [ ]:
SERVICE_URL = "YOUR_SERVICE_URL_HERE"  # Replace with your Cloud Run service URL

You'll need to first create a Session

In [ ]:
user_id = "test_user_123"
session_data = {"state": {"preferred_language": "English", "visit_count": 1}}

session_url = f"{SERVICE_URL}/apps/app/users/{user_id}/sessions"
headers = {"Content-Type": "application/json", "Authorization": f"Bearer {ID_TOKEN}"}

session_response = requests.post(session_url, headers=headers, json=session_data)
print(f"Session creation status code: {session_response.status_code}")
print(f"Session creation response: {session_response.json()}")
session_id = session_response.json()["id"]

Then you will be able to send a message

In [ ]:
message_data = {
    "app_name": "app",
    "user_id": user_id,
    "session_id": session_id,
    "new_message": {"role": "user", "parts": [{"text": "Hello! Weather in New york?"}]},
    "streaming": True,
}

message_url = f"{SERVICE_URL}/run_sse"
message_response = requests.post(
    message_url, headers=headers, json=message_data, stream=True
)

print(f"Message send status code: {message_response.status_code}")

# Print streamed response
for line in message_response.iter_lines():
    if line:
        line_str = line.decode("utf-8")
        if line_str.startswith("data: "):
            event_json = line_str[6:]
            event = json.loads(event_json)
            print(f"Received event: {event}")

### Local Testing

> You can run the application locally via the `make local-backend` command.

#### Create a session
 Create a new session with user preferences and state information


In [ ]:
user_id = "test_user_123"
session_data = {"state": {"preferred_language": "English", "visit_count": 1}}

session_url = f"http://127.0.0.1:8000/apps/app/users/{user_id}/sessions"
headers = {"Content-Type": "application/json"}

session_response = requests.post(session_url, headers=headers, json=session_data)
print(f"Session creation status code: {session_response.status_code}")
print(f"Session creation response: {session_response.json()}")
session_id = session_response.json()["id"]

#### Send a message
Send a message to the backend service and receive a streaming response


In [ ]:
message_data = {
    "app_name": "app",
    "user_id": user_id,
    "session_id": session_id,
    "new_message": {"role": "user", "parts": [{"text": "Hello! Weather in New york?"}]},
    "streaming": True,
}

message_url = "http://127.0.0.1:8000/run_sse"
message_response = requests.post(
    message_url, headers=headers, json=message_data, stream=True
)

print(f"Message send status code: {message_response.status_code}")

# Print streamed response
for line in message_response.iter_lines():
    if line:
        line_str = line.decode("utf-8")
        if line_str.startswith("data: "):
            event_json = line_str[6:]
            event = json.loads(event_json)
            print(f"Received event: {event}")